In [ ]:
import h5_helpers as h5
import dummydata
from IPython.display import display

filename = 'test.hdf5'

## Save Dummy Data for all elements into the HDF5 file


In [ ]:
print("Generating Dummy h5 file with the following settings")
run_settings = dummydata.defaults
display(run_settings)
dummydata.fill_hdf(run_settings)

## Preview of csv spectrometer data

In [1]:
import os
import pandas as pd
dir = "/Users/calum/git/Glasgow/sampleData/Beer x Bitter"
testfile = "HAN24_Sensor10_BeerBitter1_Rotation1.txt"
df = pd.read_csv(os.path.join(dir, testfile), sep='\t')

#label columns
reps = len(df.columns)-1
col_names = ['wavelength']
for r in range(reps):
	col_names.append(F"rep{r+1}")
df.columns = col_names

# datafile = 'dataframe'

# # compression_options = dict(method='zip', archive_name=f'{datafile}.csv')
# df.to_csv(f'{datafile}.csv', index=False, sep='\t')
# df2 = pd.read_csv("dataframe.csv", sep='\t')




In [2]:
from datetime import datetime
import csv_helpers as csv

dummy_metadata = {
    'sensor'            : 'HAN24',
    'element'           : 'A02',
    'fluid'             : 'beer',
}


csv.store(df, dummy_metadata)


Warning, file exists: ./raw/211102-HAN24-beer-A02.tsv, appending columns
Warning, 211102-HAN24-beer-A02 already exists in ./index.tsv, updating rep count to 15


In [ ]:
dummy_metadata = {
    'sensor'            : 'HAN24',
    'element'           : 'A02',
    'fluid'             : 'beer',
}
csv.store(df, dummy_metadata)

## Import from .txt or .csv files

### Input File

Requires a text file with at least 2 columns.
(They don't need to be named in the file)

[wavelength]	[transmission data 1]	...[transmission data n (optional)]

The default is for tab separation, but can be specified with:

separator = '\t'

### Regex
A regex must be provided to identify these metadata fields from the filename:
* sensor
* element
* fluid

Optional, will be preserved in the metadata:
* rotation


In [ ]:
#Folder containing text/csv files to import
# import_dir = "/Users/calum/git/Glasgow/sampleData/Beer x Bitter"
import_dir = "/Users/calum/git/Glasgow/sampleData/combined"

# Construct a regex to extract metadata from the filename
# example 'HAN24_Sensor9_BeerBitter3_Rotation2.txt'
regex = '(?P<sensor>.+)_Sensor(?P<element>.+)_(?P<fluid>.+)_Rotation(?P<rotation>.+).txt'

h5.import_dir_to_hdf(import_dir, regex, filename, append=False)

## Inspect an HDF5 file by metadata

In [ ]:
h5.inspect('test.hdf5')

## Example of searching by metadata

filter_by_metadata() returns a list of h5 nodes that match the
criteria.

It can search a full h5 file, or if a nodelist is provided, it will work from
that.

Search criteria must include a metadata key, and can optionally include a
metadata value

In [ ]:
e01 = h5.filter_by_metadata(filename, 'element', "01")
e01_BeerBitter1 = h5.filter_by_metadata(filename, 'fluid', "BeerBitter1", nodelist=e01)

print(e01_BeerBitter1)


## Export Table

export_dataframes() requires a list of measurements to export

e.g. by filtering for nodes with 'HAN24' in the 'sensor' metadata



In [ ]:
measurements = h5.filter_by_metadata(filename, 'sensor', 'HAN24')
export = h5.export_dataframes(filename, measurements, outfile=None)
display(export)

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
dir = "/Users/calum/git/Glasgow/sampleData/Beer x Bitter"
testfile = "HAN24_Sensor10_BeerBitter1_Rotation1.txt"
df = pd.read_csv(os.path.join(dir, testfile), sep='\t')

#label columns
reps = len(df.columns)-1
col_names = ['wavelength']
for r in range(reps):
	col_names.append(F"rep{r+1}")
df.columns = col_names
df

def filter_df(df, wl_min, wl_max, resample):

    df = df.loc[lambda dfn: dfn['wavelength'] > wl_min-1, :]
    df = df.loc[lambda dfn: dfn['wavelength'] < wl_max+1, :]

    wavel_new = np.arange(wl_min, wl_max, resample)
    result = {}
    for col in df:
        if col == 'wavelength':
            result[col] = wavel_new
        else: 
            maxval = df[col].max()
            df[col] = df[col] / maxval
            f = interp1d(df['wavelength'], df[col], 'linear')
        
            result[col] = f(wavel_new)
    return pd.DataFrame(result)



df = filter_df(df, wl_min=540, wl_max=730, resample=0.3)
df





